In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
# Load S&P 500 df
df_quake_sp500 = pd.read_csv("https://raw.githubusercontent.com/labs13-quake-viewer/ds-data/master/" +
                             "S&P%20500%20Price%20Change%20by%20Earthquake.csv", index_col=0)
df_quake_sp500.shape

(1870, 9)

In [3]:
df_quake_sp500.head()

,Date,Mag,Price_Day_0,Price_Day_7,Price_Day_14,Price_Day_30,Appr_Day_7,Appr_Day_14,Appr_Day_30
0,1950-01-30,6.8,17.020000,17.320000,17.059999,17.240000,1.762632,0.235012,1.292597
1,1950-02-02,6.9,17.230000,17.280001,16.990000,17.320000,0.290197,-1.392919,0.522345
2,1950-02-03,6.7,17.290001,17.240000,17.150000,17.320000,-0.289190,-0.809722,0.173505
3,1950-02-28,7.7,17.219999,17.200001,17.250000,17.299999,-0.116132,0.174222,0.464576
4,1950-03-07,6.7,17.200001,17.250000,17.450001,17.780001,0.290692,1.453488,3.372093


In [0]:
dates = []
for i in df_quake_sp500.Date:
  dates.append(int(''.join(c for c in i if c.isdigit())))

In [0]:
df_quake_sp500["magg"] = (df_quake_sp500["Mag"] * 10).astype(int)

In [0]:
df_quake_sp500["dates"] = dates

In [7]:
df_quake_sp500.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1870 entries, 0 to 1869
Data columns (total 11 columns):
Date            1870 non-null object
Mag             1870 non-null float64
Price_Day_0     1870 non-null float64
Price_Day_7     1870 non-null float64
Price_Day_14    1870 non-null float64
Price_Day_30    1870 non-null float64
Appr_Day_7      1870 non-null float64
Appr_Day_14     1870 non-null float64
Appr_Day_30     1870 non-null float64
magg            1870 non-null int64
dates           1870 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 175.3+ KB


In [8]:
y = df_quake_sp500['Appr_Day_30'].astype(str)
X = df_quake_sp500[['dates', 'Mag']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
print("Original shape:", X.shape, "\n")

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Original shape: (1870, 2) 

X_train shape: (1402, 2)
X_test shape: (468, 2)
y_train shape: (1402,)
y_test shape: (468,)


In [9]:
X_train.sample()

,dates,Mag
50,19520319,7.3


In [0]:
# Instantiate model with various parms
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree',
                    nrounds = 'min.error.idx', num_class = 4,
                    maximize = False, eval_metric = 'merror', eta = .2,
                    max_depth = 16, colsample_bytree = .4, nthread = -1)

In [11]:
%%time
# Train model on training data
xgb.fit(X_train, y_train)

CPU times: user 4min 51s, sys: 838 ms, total: 4min 52s
Wall time: 2min 28s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.2,
              eval_metric='merror', gamma=0, learning_rate=0.1,
              max_delta_step=0, max_depth=16, maximize=False,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nrounds='min.error.idx', nthread=-1, num_class=4,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [12]:
%%time
# Use xgb's predict method on test data
predictions = xgb.predict(X_test)

CPU times: user 1min 27s, sys: 58.3 ms, total: 1min 27s
Wall time: 45.2 s


In [0]:
# Calculate absolute errors
errors = abs(predictions.astype('float64') - y_test.astype('float64'))

In [14]:
# Print out mean absolute error
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 4.43 degrees.


In [16]:
%%time
# Calculate and display accuracy
accuracy = errors.sum() / y_test.astype('float64').sum()
print("For S&P 500, Incident Mag >= 6.7 ({} incidents)".format(df_quake_sp500.shape[0]))
print("XGB Model Score:", xgb.score(X_train, y_train))
print('XGB Model Predictive Accuracy:', round(accuracy, 2), '%.')

For S&P 500, Incident Mag >= 6.7 (1870 incidents)
XGB Model Score: 0.1818830242510699
XGB Model Predictive Accuracy: 7.28 %.
CPU times: user 4min 29s, sys: 133 ms, total: 4min 29s
Wall time: 2min 18s
